In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix, f1_score

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/Tweets.csv')
df.head()

In [ ]:
df.shape

# ****Data Preprocessing****

In [ ]:
#Check for NaN values in each column
df.isna().sum()

In [ ]:
#Percentage of NaN values
((len(df)-df.count())/len(df))*100

In [ ]:
del df['airline_sentiment_gold']
del df['negativereason_gold']
del df['tweet_coord']

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
#Sentiment Count 
sentiment_count = df.airline_sentiment.value_counts()
#Airline Review Count
airline_total = df['airline'].value_counts()

In [ ]:
index = [1,2,3]
plt.figure(1,figsize=(20,10))
plt.subplot(221)
plt.bar(index,sentiment_count,color=['red','blue','green'])
plt.xticks(index,['negative','neutral','positive'],rotation=0)
plt.xlabel('Sentiment Type')
plt.ylabel('Sentiment Count')
plt.title('Count of Type of Sentiment')
Index=[1,2,3,4,5,6]
my_colors = 'rgbkym'
plt.subplot(222)
plt.bar(Index,airline_total,color=my_colors)
plt.xticks(Index,['United','US Airways','American','Southwest','Delta','Virgin America'],rotation=90)
plt.xlabel('Airline')
plt.ylabel('Review Count')
plt.title('Airline Review Count')

From the plot it is observed that the majority of the sentiments are negative sentiments.  

In [ ]:
airline_count = df.groupby('airline')['airline_sentiment'].value_counts()

In [ ]:
def plot_sentiment_airline(airline):
    df_airline = df[df['airline']==airline]
    count = df_airline['airline_sentiment'].value_counts()
    index = [1,2,3]
    plt.bar(index,count,color=['red','blue','green'])
    plt.xticks(index,['negative','neutral','positive'],rotation=0)
    plt.xlabel('Sentiment Type')
    plt.ylabel('Sentiment Count')
    plt.title('Count of Sentiment Type of '+airline)
airlines = ['US Airways','Virgin America','United','Delta','American','Southwest']
for i in range(len(airlines)):
    plt.figure(1,figsize=(20,12))
    temp = 231+i
    plt.subplot(temp)
    plot_sentiment_airline(airlines[i])

*  Negative sentiment of United is higher compared to negative sentiment of other airlines. 
* US Airways, American, Southwest have high number of negative sentiments. 
* Delta and Virgin America's reviews are evenly balanced. 

In [ ]:
timezone_count = df['user_timezone'].value_counts()
tweet = df.groupby(['airline','airline_sentiment'])['user_timezone'].value_counts()

In [ ]:
# Airlines' Negative Sentiment Count by Date
df['tweet_created']=pd.to_datetime(df['tweet_created'])
df['tweet_created'] = df['tweet_created'].dt.date
day = df.groupby(['tweet_created','airline'])['airline_sentiment'].value_counts(sort=True)

In [ ]:
date = day.loc(axis=0)[:,:,'negative']
date.groupby(['tweet_created','airline']).sum().unstack().plot(kind='bar',figsize=(15,5))
plt.xlabel('Date')
plt.ylabel('Negative Sentiment Count')
plt.title("Airlines' Negative Sentiment Count by Date")
plt.show()

*  It's observed that on 23 February 2015 and 24 February 2015, American Airlines had the highest number of negative reviews. A sudden surge in negative reviews was observed. 
* From 2015-02-23 to 2015-02-24, the negative reviews for American airlines reduced to almost half. 
* Delta and Southwest overall have a fairly low amount of negative reviews. 
* A surge in negative reviews for airlines is observed at the end of the week. 

In [ ]:
#Number of Unique Negative Reasons 
df['negativereason'].nunique()

In [ ]:
#Negative Reason Count
nr_count = df['negativereason'].value_counts()
nr_dict = dict(df['negativereason'].value_counts())

In [ ]:
nr = ["Customer Service Issue","Late Flight","Can't Tell", "Cancelled Flight", "Lost Luggage", "Bad Flight","Flight Booking Problems",         
"Flight Attendant Complaints","longlines","Damaged Luggage"]

def plot_negativereason_count(reason,reason_count):
    index=list(range(10))
    plt.figure(figsize=(15,10))
    plot_colors = 'rgbykcm'
    plt.bar(index,reason_count,color=plot_colors)
    plt.xticks(index,reason,rotation=90)
    plt.xlabel('Negative Reason Type')
    plt.ylabel('Reason Count')
    plt.title('Count of Negative Reasons Type')
plot_negativereason_count(nr,nr_count)

* Customer Service Issue is the main reason[](http://) for a negative sentiment followed by Late Flight.
* Damaged Luggage is observed to be the least probable reason for which a negative sentiment is observed. 

In [ ]:
negative_df = df.groupby('airline')['negativereason'].value_counts(ascending=False)

In [ ]:
#Negative Reason Count for Airlines
negative_df.groupby(['airline','negativereason']).sum().unstack().plot(kind='bar',figsize=(20,10))
plt.xlabel('Airline')
plt.ylabel('Negative Reason Count')
plt.title("Negative Reason Count for Airlines")
plt.show()

* It is observed that American, US Airways, and, United have the highest negative reviews for **Customer Service Issue** making this reason the main concern for airlines. 
* The top negative reason for all airlines except Delta is Customer Service Issue. 
* The least popular reason for negative reviews for all airlines except Virgin America is Damaged Luggage. 

## Preprocessing text of the tweets

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def text_to_words(tweet):
    letters = re.sub("^a-zA-Z"," ",tweet)
    words = letters.lower().split()
    stops = set(stopwords.words("english"))
    meaningful_words = [w for w in words if not w in stops]
    return( " ".join(meaningful_words))

In [ ]:
df['sentiment'] = df['airline_sentiment'].apply(lambda x: 0 if x=='negative' else 1)
df['text_clean'] = df['text'].apply(lambda x:text_to_words(x))

In [ ]:
#Train-test split
train,test=train_test_split(df,test_size=0.2,random_state=42)

The dataset is split into training and testing dataset in the 80:20 ratio.

In [ ]:
def clean_tweet(tweet_text):
    res = []
    for tweet in tweet_text:
        res.append(tweet)
    return res
train_clean_text = clean_tweet(train['text_clean'])
test_clean_text = clean_tweet(test['text_clean'])

In [ ]:
counter_vectorizer = CountVectorizer(analyzer = "word")
train_features= counter_vectorizer.fit_transform(train_clean_text)
test_features=counter_vectorizer.transform(test_clean_text)

In [ ]:
#Classifiers
Classifiers = [
    SVC(kernel="rbf", C=0.025, gamma = 'scale'),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    GradientBoostingClassifier(),GaussianNB()]

In [ ]:
dense_features=train_features.toarray()
dense_test= test_features.toarray()
Accuracy=[]
Model=[]
for clf in Classifiers:
    try:
        fit = clf.fit(train_features,train['sentiment'])
        pred = fit.predict(test_features)
    except Exception:
        fit = clf.fit(dense_features,train['sentiment'])
        pred = fit.predict(dense_test)
    accuracy = accuracy_score(pred,test['sentiment'])
    Accuracy.append(accuracy)
    Model.append(clf.__class__.__name__)
    print('Accuracy of '+clf.__class__.__name__+' is '+str(accuracy)) 
    print(classification_report(pred,test['sentiment']))
    
    #Confusion Matrix
    cm = confusion_matrix(pred,test['sentiment'])
    plt.figure()
    plot_confusion_matrix(cm,cmap=plt.cm.Blues)
    plt.xticks(range(2), ['Negative', 'Positive'],color='black')
    plt.yticks(range(2), ['Negative', 'Positive'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

**Random Forest Classifier** has the highest accuracy as well as the f-1 score. 

In [ ]:
index=list(range(len(Classifiers)))
plt.bar(index,Accuracy,color='rgbyk')
plt.xticks(index,Model,rotation=90)
plt.ylabel('Accuracy')
plt.xlabel('Model')
plt.title('Classifier Accuracies')

It is observed that **Random Forest Classifier** performs better than the other classifiers for the given dataset. 
**Random Forest Classifier** has the highest accuracy among all the algorithms. 